# Apply YOLO style Model For OCR

## An effectient one-shot deep detwork for OCR

After some searching, I didn't find any dataset with image+character label+ single char level bounding box。

So the most direct solution: Create a scene text detection data generator

In [1]:
from torch.utils.data import dataset
from torch.utils.data import DataLoader
from constant import *
import numpy as np

from PIL import Image


### Texted image generator

In [2]:
from glob import glob
from PIL import Image
from PIL import ImageFont
from PIL import ImageDraw 
import os


In [4]:
rg_n,rg_l,rg_u,rg_c,

((0, 10), (10, 36), (36, 62), (62, 3566))

In [5]:
len(chars)

3566

In [6]:
from glob import glob
from PIL import Image
from PIL import ImageFont
from PIL import ImageDraw 
import os

fonts=glob("/data/fonts_cn/*")

def rd(scale,start=0):
    return int(np.random.rand()*scale)+start

def rd_font(size):
    return ImageFont.FreeTypeFont(fonts[rd(len(fonts))],size)

class Make_Char(dataset.Dataset):
    def __init__(self,**kwargs):
        """
        kwargs
        img_dir
        forge_dir
        """
        super(Make_Char,self).__init__()
        for k, v in kwargs.items():
            setattr(self, k, v)
        if self.img_dir[-1]!="/":
            self.img_dir+="/"
        if self.forge_dir[-1]!="/":
            self.forge_dir+="/"
        self.img_list = glob(self.img_dir+"*")
        self.df_dicts = [dict()]*len(self.img_list)
        
    def __len__(self):
        return len(self.img_list)
    
    def __getitem__(self, idx):
        fn_id = self.img_list[idx].split("/")[-1]
        fn = self.id2url(fn_id)
        img = Image.open(fn).convert("RGB").resize((WIDTH,HEIGHT))
        draw = ImageDraw.Draw(img)
        nb_ttl = rd(10,15)
        bbox=list()
        category_id=list()
        for nb,rg in zip([.4,.2,.2,.2],[rg_n,rg_l,rg_u,rg_c]):
            nb_ = int(nb*nb_ttl)
            bbox_,cateid_=self.write_char(draw,nb_,rg)
            bbox += bbox_
            category_id += cateid_
        filename=[fn_id]*len(category_id)
        self.df_dicts[idx] = pd.DataFrame({"bbox":bbox,"filename":filename,"image_id":filename,"category_id":category_id})
        img.save(self.forge_dir+fn_id)
        return 0
        
    def id2url(self, filename):
        return self.img_dir + filename
    
    def write_char(self,draw,nb_c,rg):
        bbox =list()
        category_id =list()
        for i in range(nb_c):
            scale = rd(30,20)
            x,y=rd(WIDTH - scale), rd(HEIGHT - scale)
            f_color = (rd(255),rd(255),rd(255),rd(200,50))
            ft = rd_font(rd(30)) # font
            cate_id = np.random.choice(range(rg[0],rg[1]))
            txt = chars[cate_id]
            f_size = ft.getsize(text=txt)
            
            draw.text((x,y),txt,f_color,font=ft)
            
            bbox.append([x,y,f_size[0],f_size[1]])
            category_id.append(cate_id)
            
        return bbox,category_id
            

In [7]:
%rm /data/forge/char_detect/*
mc = Make_Char(img_dir = "/data/coco/val2017/",forge_dir="/data/forge/char_detect/")
dl = DataLoader(mc,batch_size=4,shuffle=True)
mc_gen=iter(dl)

from tqdm import trange
t=trange(len(dl))
for i in t:_ = next(mc_gen)

rm: /data/forge/char_detect/*: No such file or directory


100%|██████████| 1250/1250 [07:25<00:00,  2.81it/s]


In [10]:
pd.concat(mc.df_dicts,axis=0)

,bbox,category_id,filename,image_id
0,"[250, 210, 7, 12]",8,000000000139.jpg,000000000139.jpg
1,"[248, 154, 19, 35]",2,000000000139.jpg,000000000139.jpg
2,"[13, 253, 5, 13]",5,000000000139.jpg,000000000139.jpg
3,"[237, 178, 11, 17]",1,000000000139.jpg,000000000139.jpg
4,"[95, 231, 11, 25]",5,000000000139.jpg,000000000139.jpg
5,"[225, 103, 4, 7]",4,000000000139.jpg,000000000139.jpg
6,"[106, 225, 5, 16]",19,000000000139.jpg,000000000139.jpg
7,"[152, 242, 17, 35]",22,000000000139.jpg,000000000139.jpg
8,"[25, 183, 9, 30]",26,000000000139.jpg,000000000139.jpg
9,"[152, 121, 6, 9]",51,000000000139.jpg,000000000139.jpg
